In [2]:
# Install dependencies.
# !pip install asyncio==3.4.3 asyncpg==0.27.0 cloud-sql-python-connector["asyncpg"]==1.2.3
# !pip install numpy==1.22.4 pandas==1.5.3
# !pip install pgvector==0.1.8
# !pip install langchain==0.0.196 transformers
# !pip install google-cloud-aiplatform==1.26.0

In [3]:
# Automatically restart kernel after installs so that your environment
# can access the new packages.
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [12]:
import os
import torch
# Please fill in these values.
project_id = "engaged-plasma-439804-k0"  # @param {type:"string"}
database_password = os.getenv('DATABASE_PASSWORD')  # @param {type:"string"}
region = "us-central1"  # @param {type:"string"}
instance_name = "research-paper-db"  # @param {type:"string"}
database_name = "research-papers"  # @param {type:"string"}
database_user = "superuser"  # @param {type:"string"}


# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"
assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"

In [ ]:
# @markdown Verify that you are able to connect to the database. Executing this block should print the current PostgreSQL server version.

import asyncio
import asyncpg
from google.cloud.sql.connector import Connector


async def test_connection():
    # get current running event loop to be used with Connector
    print(1)
    loop = asyncio.get_running_loop()
    print(1)
    # initialize Connector object as async context manager
    async with Connector(loop=loop) as connector:
        print(1)
        # create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}"
            # ... additional database driver args
        )

        # query Cloud SQL database
        results = await conn.fetch("SELECT version()")
        print(results[0]["version"])

        # close asyncpg connection
        await conn.close()


# Test connection with `asyncio`
await test_connection()  # type: ignore

In [64]:
# Save the Pandas dataframe in a PostgreSQL table.

import asyncio
import asyncpg
from google.cloud.sql.connector import Connector


async def reset_database():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await conn.execute("DROP TABLE IF EXISTS papers CASCADE")
        # Create the `products` table.
        await conn.execute(
            """CREATE TABLE papers(
                                doi TEXT PRIMARY KEY,
                                title TEXT,
                                abstract TEXT,
                                content TEXT,
                                citation_text TEXT,
                                embedding vector(1024))"""
        )

        # Copy the dataframe to the `products` table.
        # tuples = list(df.itertuples(index=False))
        # await conn.copy_records_to_table(
        #     "products", records=tuples, columns=list(df), timeout=10
        # )
        await conn.close()


# Run the SQL commands now.
# await reset_database()  # type: ignore

In [4]:
# Store the generated vector embeddings in a PostgreSQL table.
# This code may run for a few minutes.

import asyncio
import asyncpg
from google.cloud.sql.connector import Connector
import numpy as np
from pgvector.asyncpg import register_vector
import pandas as pd


async def insert_into_papers(papers_df = None):
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await conn.execute("CREATE EXTENSION IF NOT EXISTS vector")
        await register_vector(conn)


        # res = await conn.execute("DROP TABLE IF EXISTS papers CASCADE")
        # print(res)
        # Create the `products` table.
        # await conn.execute(
        #     """CREATE TABLE papers(
        #                         doi TEXT PRIMARY KEY,
        #                         title TEXT,
        #                         abstract TEXT,
        #                         content TEXT,
        #                         citation_text TEXT,
        #                         embedding vector(1024))"""
        # )
        if type(df) == pd.DataFrame:
            # Store all the generated embeddings back into the database.
            insert_values = [
                (
                    str(row["doi"]),
                    row["title"],
                    row["abstract"],
                    row["content"],
                    row["citation_text"],
                    np.array(row["embedding"]),
                    row['id'],
                    np.array(row['title_embedding']),

                )
                for index, row in papers_df.iterrows()
            ]

            # Use executemany for bulk insert
            await conn.executemany(
                "INSERT INTO papers (doi, title, abstract, content, citation_text, embedding, id, title_embedding) VALUES ($1, $2, $3, $4, $5, $6, $7, $8)",
                insert_values
            )

        await conn.close()

# import pandas as pd
# import numpy as np
# df = pd.read_csv("test.csv")
# df["embedding"] = [np.random.rand(1024) for _ in range(len(df))]
# df
# # Run the SQL commands now.
# await insert_into_papers(df)  # type: ignore

In [56]:
async def update_papers(papers_df = None):
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await conn.execute("CREATE EXTENSION IF NOT EXISTS vector")
        await register_vector(conn)

        if type(papers_df) == pd.DataFrame:
            # Store all the generated embeddings back into the database.
            update_values = [
                (
                    str(row["doi"]),
                    row["PageRank"],
                )
                for index, row in papers_df.iterrows()
            ]

            # Use executemany for bulk insert
            await conn.executemany(
                """
                UPDATE papers 
                SET relevance = $2
                WHERE doi = $1
                """,
                update_values
            )

        await conn.close()

In [6]:
async def fetch(embeding, nbr_articles=2, DESC=True):
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)
        # Find similar products to the query using cosine similarity search
        # over all vector embeddings. This new feature is provided by `pgvector`.
        results = await conn.fetch(
            f"""
                            SELECT title, 1 - (embedding <=> $1) AS similarity, abstract, doi, 1 - (title_embedding <=> $1) AS title_similarity
                            FROM papers
                            ORDER BY similarity {"DESC" if DESC else ""}
                            LIMIT $2
                            """,
            embeding,
            nbr_articles
        )

        if len(results) == 0:
            raise Exception("Did not find any results. Adjust the query parameters.")
        matches = []
        for r in results:
            # Collect the description for all the matched similar toy products.
            matches.append(
                {
                    "title": r["title"],
                    "sim": r["similarity"],
                    "title_similarity": r["title_similarity"],
                    "abstract": r["abstract"],
                }
            )

        await conn.close()
        return matches
if False:
    embeding = np.random.rand(768)

    # Run the SQL commands now.
    res = await fetch(embeding)  # type: ignore
    res

In [ ]:
from transformers import AutoTokenizer, AutoModel
model_type = 'jina'
if model_type == "bert":
    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
elif model_type == "roberta":
    tokenizer = AutoTokenizer.from_pretrained('FacebookAI/roberta-base')
    model = AutoModel.from_pretrained('FacebookAI/roberta-base')  
elif model_type == 'jina':
    tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v3')
    model = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True)
else:
    Exception("No model chosen")

In [ ]:
import torch
# Sample text
text = "SciBERT is a pretrained transformer model for scientific text."

# Tokenize the input text
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

# Forward pass to get the embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract the last hidden states (embeddings)
last_hidden_states = outputs.last_hidden_state

# Optionally, get the embeddings for the [CLS] token
cls_embedding = last_hidden_states[:, 0, :]

# Print the shape of the embeddings
print("Shape of last hidden states:", last_hidden_states.shape)
print("Shape of CLS embedding:", cls_embedding.shape)


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("test.csv")
df["embedding"] = [np.random.rand(768) for _ in range(len(df))]
with torch.no_grad():
    embeds = model(**tokenizer(df["abstract"].to_list(), return_tensors='pt', padding=True, truncation=True)).last_hidden_state[:, 0, :]
    for idx, emb in enumerate(embeds):
        df.at[idx, "embedding"] = emb.numpy()
df

In [103]:
# await reset_database() #[-0.1714102, 0.548595, -0.22505158, 0.003960
# await insert_into_papers(df) #[-0.1714096, 0.5485956, -0.22505

In [ ]:
async def fetch_all():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)
        # Find similar products to the query using cosine similarity search
        # over all vector embeddings. This new feature is provided by `pgvector`.
        results = await conn.fetch(
            """
                            SELECT *
                            FROM papers
                            """
        )

        if len(results) == 0:
            return []
        matches = []
        for r in results:
            # Collect the description for all the matched similar toy products.
            matches.append(
                {
                    "doi": r["doi"],
                    "title": r["title"],
                    "abstract": r["abstract"],
                    "content": r["content"],
                    "emb": r["embedding"],
                    "id": r["id"],
                }
            )

        await conn.close()
        return matches
matches = await fetch_all()
len(matches)

In [ ]:
df_abstract = pd.read_csv("./data/ml_100_abstracts.csv")
df_abstract["doi"] = None
df_abstract["title"] = None
df_abstract["content"] = "None"
df_abstract["citation_text"] = "None"
for i in range(len(df_abstract)):
    df_abstract.loc[i, "doi"] = "paper" + str(i)
    df_abstract.loc[i, "title"] = str(i)
    df_abstract.loc[i, "content"] = str(i)
    df_abstract.loc[i, "citation_text"] = str(i)
df_abstract.rename(columns={'Abstract': 'abstract'}, inplace=True)
df_abstract

In [10]:
import pandas as pd
def add_embedings(df: pd.DataFrame, model, tokenizer, field_to_embedd="abstract", embedding_field_name="embedding"):
    # df["embedding"] = None
    with torch.no_grad():
        inputs = tokenizer(df[field_to_embedd].to_list(), return_tensors='pt', padding=True, truncation=True)
        embeds = model(**inputs, task="retrieval.passage").last_hidden_state[:, 0, :]
        df.insert(len(df), embedding_field_name, embeds.tolist())
        # assert len(df) == len(embeds)
        # for idx, emb in enumerate(embeds):
        #     df.at[idx, "embedding"] = emb.numpy()

In [ ]:
# reset_database() #[-0.1714102, 0.548595, -0.22505158, 0.003960
# await insert_into_papers(df_abstract) #[-0.1714096, 0.5485956, -0.22505

In [ ]:
text = ["summarize the papers related to linear regression",
        "quantum mechanics is a very interesting field plese get mo som epapers related to that subject",
        "What are transformers what are examples of transformer, encoder decoder models"
        ]

async def get_similar(text, nbr_articles=5):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    # Forward pass to get the embeddings
    with torch.no_grad():
        outputs = model(**inputs, task="retrieval.query")

    # Extract the last hidden states (embeddings)
    last_hidden_states = outputs.last_hidden_state

    # Optionally, get the embeddings for the [CLS] token
    cls_embedding = last_hidden_states[:, 0, :]
    return await fetch(cls_embedding[0], nbr_articles=nbr_articles, DESC=True)

for sent in text:
    print(sent)
    res = await get_similar(sent)
    for i in res:
        print(i)
    

In [33]:
df_json = pd.read_json("./data/ml_papers.json")
df_json.columns

Index(['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi',
       'report-no', 'categories', 'license', 'abstract', 'versions',
       'update_date', 'authors_parsed'],
      dtype='object')

In [42]:
df_json_filterd = df_json[['doi', 'update_date']]
df_json_filterd = df_json_filterd.drop_duplicates(subset='doi', keep='first')
df_json_filterd.isna().sum()
df_json_filterd['update_date'] = pd.to_datetime(df_json_filterd['update_date'])

In [48]:
df_rank = pd.read_csv("./data/pagerank_results.csv")
df_rank.rename(columns={'Paper': 'doi'}, inplace=True)
df_rank

,Rank,doi,PageRank
0,1,10.1007/s10458-019-09421-1,0.006333
1,2,10.3844/ajeassp.2018.1310.1319,0.006057
2,3,10.3844/ajeassp.2018.1320.1331,0.006057
3,4,10.22331/q-2021-06-29-483,0.005126
4,5,10.1103/PhysRevApplied.16.054035,0.004757
...,...,...,...
1356,1357,10.23919/FPL.2017.8056832,0.000399
1357,1358,10.1109/JPROC.2021.3052449,0.000399
1358,1359,10.1145/3341105.3374013,0.000399
1359,1360,10.1109/IJCNN48605.2020.9206730,0.000399


In [ ]:
# import warnings
# from cryptography.utils import CryptographyDeprecationWarning 
# warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
dois = [item['doi'] for item in await fetch_all()]
print(dois)
chunk_size = 5
seq = df_json_filterd[~df_json_filterd['doi'].isna()][5000:10000]
seq = seq[~seq['doi'].isin(dois)]
for idx, chunk in enumerate(chunker(seq=seq, size=chunk_size)):
    print("processing ", idx*chunk_size, "to", (idx+1)*chunk_size)
    add_embedings(chunk, model=model, tokenizer=tokenizer)
    add_embedings(chunk, model=model, tokenizer=tokenizer, field_to_embedd="title", embedding_field_name="title_embedding")
    await insert_into_papers(chunk)


In [ ]:

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
dois = [item['doi'] for item in await fetch_all() if item['id'] == None]
print(len(dois))
chunk_size = 5
seq = df_json_filterd[~df_json_filterd['doi'].isna()][5000:10000]
seq = seq[seq['doi'].isin(dois)]
for idx, chunk in enumerate(chunker(seq=seq, size=chunk_size)):
    print("processing ", idx*chunk_size, "to", (idx+1)*chunk_size)
    add_embedings(chunk, model=model, tokenizer=tokenizer, field_to_embedd="title", embedding_field_name="title_embedding")
    await update_papers(chunk)

In [ ]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
chunk_size = 500
for idx, chunk in enumerate(chunker(seq=seq, size=chunk_size)):
    print("processing ", idx*chunk_size, "to", (idx+1)*chunk_size)
    
    await update_papers(chunk)

In [55]:

chunk_size = 500
seq = df_json_filterd[~df_json_filterd['doi'].isna()][:10000]
seq = seq.merge(df_rank, on='doi', how='left')
seq['PageRank'] = seq['PageRank'].fillna(0)
seq

,doi,update_date,Rank,PageRank
0,10.1109/ITW.2007.4313111,2016-11-15,NaN,0.000000
1,10.1109/TSP.2008.920143,2009-11-13,782.0,0.000399
2,10.1143/JPSJ.76.114001,2009-11-13,783.0,0.000399
3,10.1109/ICTAI.2007.99,2016-11-15,NaN,0.000000
4,10.1371/journal.pcbi.0030252,2010-02-22,NaN,0.000000
...,...,...,...,...
9995,10.1109/JBHI.2020.2984907,2020-10-15,NaN,0.000000
9996,10.1145/3394231.3397910,2020-10-15,NaN,0.000000
9997,10.1145/3442381.3449898,2021-04-02,NaN,0.000000
9998,10.1145/3385003.3410921,2020-10-15,NaN,0.000000


In [58]:
async def fetch_all_citations():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)
        # Find similar products to the query using cosine similarity search
        # over all vector embeddings. This new feature is provided by `pgvector`.
        results = await conn.fetch(
            """
            SELECT *
            FROM citations
            """
        )

        if len(results) == 0:
            return []
        matches = []
        for r in results:
            # Collect the description for all the matched similar toy products.
            matches.append(
                {
                    "source_paper": r["source_paper"],
                    "cited_by": r["cited_by"],
                }
            )

        await conn.close()
        return matches
await fetch_all_citations()

c:\Users\User\GaTech\hackathon\i-cite\venv\Lib\site-packages\google\cloud\sql\connector\instance.py:330: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  expiration = x509.not_valid_after


[{'source_paper': '10.1038/s41467-020-14454-2',
  'cited_by': '10.1088/2058-9565/ab4eb5'},
 {'source_paper': '10.1007/978-3-030-22368-7_26',
  'cited_by': '10.1007/s10851-019-00915-x'},
 {'source_paper': '10.5220/0007581508300836',
  'cited_by': '10.1007/s11063-021-10490-1'},
 {'source_paper': '10.1002/sam.11461',
  'cited_by': '10.1016/j.eswa.2020.113680'},
 {'source_paper': '10.1016/j.sigpro.2020.107486',
  'cited_by': '10.1109/TCI.2020.3006718'},
 {'source_paper': '10.1109/LCSYS.2019.2921158',
  'cited_by': '10.1109/TAC.2022.3159453'},
 {'source_paper': '10.1109/LCSYS.2019.2921158',
  'cited_by': '10.1109/TAC.2021.3112851'},
 {'source_paper': '10.1109/ACCESS.2019.2903130',
  'cited_by': '10.1109/ACCESS.2019.2925569'},
 {'source_paper': '10.1039/C9ME00039A',
  'cited_by': '10.1088/2632-2153/aba947'},
 {'source_paper': '10.1109/PRIA.2019.8786013',
  'cited_by': '10.1016/j.eswa.2019.03.040'},
 {'source_paper': '10.1109/PRIA.2019.8786013',
  'cited_by': '10.1109/PRIA.2019.8785979'},
 {'

In [30]:
def bfs(bois: list[str], citations: list[dict], max_papers=10) -> list[str]:
    """
    Performs a breadth-first search (BFS) to explore related papers based on citations.

    This function takes a list of base of interest (BOI) papers and a list of citation data to find
    additional papers that are cited by or cite the BOI papers. The search continues until it reaches 
    a specified maximum number of papers.

    Parameters:
    - bois (list[str]): A list of DOIs (or identifiers) representing the base papers of interest.
    - citations (list[dict]): A list of citation records, where each record is a dictionary containing:
        - 'cited_by' (str): The DOI of the paper that is citing another paper.
        - 'source_paper' (str): The DOI of the paper being cited.
    - max_papers (int): The maximum number of papers to return, including the base papers and 
                        their related papers. Default is 10.

    Returns:
    - list[str]: A list of DOIs representing the base papers and any additional papers found 
                 through citations, up to the maximum specified.

    Note:
    The search stops once the total number of identified papers reaches the `max_papers` limit.
    """
    based_on = set()
    future_work = set()
    queue = [i for i in bois]
    for boi in queue:
        for citation in citations:
            if citation['cited_by'] == boi and citation['source_paper'] not in based_on:
                print(citation, "by")
                queue.append(citation['source_paper'])
                based_on.add(citation['source_paper'])
            if len(bois) + len(based_on) >= max_papers: break
        if len(bois) + len(based_on) >= max_papers: break

    queue = [i for i in bois]
    for boi in queue:
        for citation in citations:
            if citation['source_paper'] == boi  and citation['cited_by'] not in future_work:
                print(citation, "source")
                queue.append(citation['cited_by'])
                future_work.add(citation['cited_by'])
            if len(bois) + len(based_on) + len(future_work) >= max_papers: break
        if len(bois) + len(based_on) + len(future_work) >= max_papers: break
    return bois + list(based_on) + list(future_work)
    


In [ ]:
bfs(["10.1109/ICASSP.2019.8683438"], await fetch_all_citations()), set(bfs(["10.1109/ICASSP.2019.8683438"], await fetch_all_citations()))

In [61]:
async def fetch_substring_match(query: str):# -> list[Paper]:
    """
    Fetches all papers from the database that have a title containing a specific substring.

    This method establishes an asynchronous connection to the Cloud SQL database, 
    executes a query to find all entries in the `papers` table where the `title` 
    column contains the specified substring (case-insensitive). The results are 
    returned as a list of `Paper` objects.

    Args:
        query (str): The substring to search for in the title of the papers.

    Returns:
        list[Paper]: A list of `Paper` objects that match the query. 
                     Returns an empty list if no matches are found.

    Raises:
        Exception: Any exceptions related to database connection or query execution 
                   will propagate to the caller.
    """
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)
        # Find similar products to the query using cosine similarity search
        # over all vector embeddings. This new feature is provided by `pgvector`.
        results = await conn.fetch(
            f"""
            SELECT *
            FROM papers
            WHERE title ILIKE '%{query}%';
            """
        )

        if len(results) == 0:
            return []
        matches = []
        for r in results:
            # Collect the description for all the matched similar toy products.
            matches.append(
                ""#Paper.from_dict(r)
            )

        await conn.close()
        return matches
await fetch_substring_match("Conditional")

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']